In [1]:
import pandas as pd
from datetime import date, timedelta
from modifiedGlicko import Team, ratingUpdate
import math
# import glicko2

init_d = 350
init_v = 0.001

[1025.4859444495833, 1043.7158238313502, 1047.8232526452093, 1051.564788862337, 1061.893931570815, 1065.0388659274918, 1076.9547207459634, 1090.9433025689025]
[1015.291566641589, 1010.7582499046903, 1006.6397764685555, 995.2799620952633, 1005.736754373586, 1005.6983550673284, 1020.5948521224575, 1006.6281666452078]
[1005.154055528046, 1041.5465941941538, 1037.4068244398945, 1018.5761956174197, 994.5080495231064, 981.6529978232321, 966.7624653385337, 952.8580871659084]
[984.708433358411, 966.4785540081228, 974.7184299971232, 1001.2594018977584, 1008.1808733031781, 995.3580631964135, 1007.233662344651, 1021.2052544498849]
[974.5140555504167, 938.0117824359066, 929.7723998853289, 948.677174590693, 945.2593625759127, 938.9059103349767, 930.0443059583689, 930.0771311234046]


In [2]:
df = pd.read_csv("bucs_results.csv")

postcodes = pd.read_csv("institute_postcodes.csv")

# print the first few rows of the DataFrame to verify that it was loaded correctly
df.head()
df = df.drop(columns=['No Score Type', 'Walkover Reason','Knockout Fixtures', 'Number of Walkovers', 'Voids', 'Blowout Game(s)', 'Zero Score?', 'Blowout to Zero', 'Team Conceding Walkover', 'Fixtures',
       'League Fixtures', 'Time'])

In [3]:
df = df[df["Fixture type"]=="League"]

In [4]:
def generate_glicko_init_rating(comp):
    if "National" in comp or "Premier" in comp:
        return 2600
    if "North/South" in comp or "(North)" in comp or "(South)" in comp:
        return 2200
    if "Tier 1" in comp:
        return 2000
    if "Tier 2" in comp:
        return 1500
    if "Tier 3" in comp:
        return 1200
    if "Tier 4" in comp:
        return 900
    if "Tier 5" in comp:
        return 600

In [5]:
df['HG_RANK_Initial'] = df.apply(lambda row: generate_glicko_init_rating(row.Competition), axis=1)
df['AG_RANK_Initial'] = df.apply(lambda row: generate_glicko_init_rating(row.Competition), axis=1)

In [6]:
team_gscores = {}

for i, team in df[df["Community"]=="BUCS Hockey 22-23"].iterrows():
# for i, team in df.iterrows():
    team_gscores[team.Home] = team.HG_RANK_Initial
    team_gscores[team.Away] = team.AG_RANK_Initial
    if math.isnan(team.HG_RANK_Initial):
        print(team.Home, team.HG_RANK_Initial, "Home", team.Competition)
    if math.isnan(team.AG_RANK_Initial):
        print(team.Away, team.AG_RANK_Initial, "Away", team.Competition)
    
#     print(team_gscores[team.Home], team_gscores[team.Away])
    
# print(team_gscores)

team_gObj = {}

for team, gscore in team_gscores.items():
    team_gObj[team] = Team(team, gscore, initial_deviation=init_d, initial_volatility=init_v)
    
# del team_gscores 

In [7]:
for i, row in df.iterrows():
    
    if row['Home'] not in team_gObj:
        team_gObj[row['Home']] = Team(row['Home'], row['HG_RANK_Initial'], initial_deviation=init_d, initial_volatility=init_v)
    home_gobj = team_gObj[row['Home']]
    
    if row['Away'] not in team_gObj:
        team_gObj[row['Away']] = Team(row['Away'], row['AG_RANK_Initial'], initial_deviation=init_d, initial_volatility=init_v)
    away_gobj = team_gObj[row['Away']]

    score_diff = int(row['Home Score']) - int(row['Away Score'])
 
    homeTeam, awayTeam = ratingUpdate(
        (home_gobj), 
        (away_gobj),
        score_diff
    )
    team_gObj[row['Home']] = homeTeam
    team_gObj[row['Away']] = awayTeam
    

# print(team_gObj)

In [12]:
for team, obj in team_gObj.items():
    if "Women's" in team:
        continue
    if "Southampton" not in team:
        continue
    
    print(team, obj.rating, obj.RD)

Southampton Men's 4 597.863207685731 85.45862097585098
Southampton Men's 3 760.6869669371483 77.13928436205885
Southampton Men's 2 1007.1694693834183 74.38446963079738
Southampton Men's 1 2096.717674886989 71.37862349090528


In [9]:
float('nan') +4

nan